In [1]:
#必要なライブラリをインポート
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import time


In [2]:
url_base = 'https://www.rakumachi.jp/syuuekibukken/area/prefecture/dimAll/?pref%5B%5D=13&limit=60&layout%5B%5D='
url_up = 'https://www.rakumachi.jp/syuuekibukken/area/prefecture/dimAll/?page='
url_middle = '&pref[]=13&limit=60&layout[]='
url_down = '&sort=property_created_at&sort_type=desc'

In [ ]:
names = []
rooms = []
prices = []
grosses = []
places = []
stations = []
years = []
totals = []
stractures = []
areas = []
floors = []

In [ ]:
for r in range(14):
    urls = []
    url = url_base + str(r+1) 
    urls.append(url)
    
    #データ取得
    result = requests.get(url)
    c = result.content

    #HTMLを元に、オブジェクトを作る
    soup = BeautifulSoup(c)
    page = int(int(soup.find('strong',{'id':'filter_count_result_bottom'}).text.replace(',','')) / 60)+1
    
    for i in range(page):
        dum = url_up + str(i+2) +url_middle + str(r+1) + url_down
        urls.append(dum)
    
    for url in urls:
        #データ取得
        result = requests.get(url)
        c = result.content

        #HTMLを元に、オブジェクトを作る
        soup = BeautifulSoup(c)

        prop = soup.find_all('div',{'class':'propertyBlock__mainArea'})

        for i in range(len(prop)):
            rooms.append(r)
            if prop[i].find('p',{'class':'propertyBlock__name'}).text != '会員登録（無料）するとこの物件をご覧頂けます':
                if prop[i].find('p',{'class':'propertyBlock__name'}) is not None:
                    names.append(prop[i].find('p',{'class':'propertyBlock__name'}).text)
                else:
                    names.append('NA')

                if  prop[i].find('strong',{'class':'price'}) is not None:
                    prices.append(prop[i].find('strong',{'class':'price'}).text)
                else:
                    prices.append('NA')

                if  prop[i].find('strong',{'class':'gross'}) is not None:
                    grosses.append(prop[i].find('strong',{'class':'gross'}).text)
                else:
                    grosses.append('NA')

                if prop[i].find_all('td')[0] is not None:
                    places.append(prop[i].find_all('td')[0].text)
                else:
                    places.append('NA')

                if prop[i].find_all('td')[1] is not None:
                    stations.append(prop[i].find_all('td')[1].text)
                else:
                    stations.append('NA')

                if prop[i].find_all('td')[2] is not None:
                    years.append(prop[i].find_all('td')[2].text)
                else:
                    years.append('NA')

                if prop[i].find_all('td')[3] is not None:
                    totals.append(prop[i].find_all('td')[3].text)
                else:
                    totals.append('NA')

                if prop[i].find_all('td')[4] is not None:
                    stractures.append(prop[i].find_all('td')[4].text)
                else:
                    stractures.append('NA')

                if prop[i].find_all('td')[5] is not None:
                    areas.append(prop[i].find_all('td')[5].text)
                else:
                    areas.append('NA')

                if prop[i].find_all('td')[6] is not None:
                    floors.append(prop[i].find_all('td')[6].text)
                else:
                    floors.append('NA')
            else:
                names.append('会員用専用')
                prices.append('NA')
                grosses.append('NA')
                places.append('NA')
                stations.append('NA')
                years.append('NA')
                totals.append('NA')
                stractures.append('NA')
                areas.append('NA')
                floors.append('NA')

        #プログラムを10秒間停止する（スクレイピングマナー）
        time.sleep(10)

C:\Users\Ryoji Nomura\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Ryoji Nomura\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [ ]:
names = Series(names)
rooms = Series(rooms)
prices = Series(prices)
grosses = Series(grosses)
places = Series(places)
stations = Series(stations)
years = Series(years)
totals = Series(totals)
stractures = Series(stractures)
areas = Series(areas)
floors = Series(floors)


In [ ]:
# replace room_type
room_type = ['1R','1K','1DK','1LDK','2K','2DK','2LDK','3K','3DK','3LDK','4K','4DK','4LDK','5DK']

room_types = rooms

for (i,r) in enumerate(room_type):
    room_types = room_types.replace(i,r)

In [ ]:
#売買価格（千円）
prices_int = prices
prices_int = prices_int.str.replace('\n              ','')
prices_int = prices_int.str.replace('万円            ','0')
prices_int = prices_int.str.replace('億円            ','00000')
prices_int = prices_int.str.replace('億','')
prices_int = prices_int.replace('NA',0).astype(int)

In [ ]:
#表面利回り
grosses_float = grosses
grosses_float = grosses_float.str.replace('\n              ','')
grosses_float = grosses_float.str.replace('%            ','')
grosses_float = grosses_float.replace('NA',0).astype(float)
grosses_float = grosses_float/100

In [ ]:
#住所
places_rev =places
places_rev =places_rev.str.replace('\n      ','')
places_rev =places_rev.str.replace('               ','')
places_rev =places_rev.str.replace('地図','')
places_rev =places_rev.str.replace('\n','')
places_rev =places_rev.str.replace('[','')
places_rev =places_rev.str.replace(']','')

In [ ]:
#平米数
area_float = areas
area_float = area_float.str.replace('㎡','')
area_float = area_float.replace('NA',0)
area_float = area_float.astype(float)


In [ ]:
#路線
route = stations.str.split('線', expand=True)[0].astype(str) + '線'

In [ ]:
#駅
station_dum = stations.str.split('線', expand=True)[1]
station_rev = station_dum.str.split('駅', expand=True)[0].astype(str) + '駅'

#駅から時間、バスは未解決
station_dum2 = station_dum.str.split('駅', expand=True)[1]
walk = station_dum2.str.split('バス', expand=True)[0]
walk_dum = walk.str.split('分', expand=True)[0]
walk_mins = walk_dum.str.split('歩', expand=True)[1].fillna(0).astype(int)

bus = station_dum2.str.split('バス', expand=True)[1]
bus = bus.str.split('分',expand=True)[0].str.split('停',expand=True)[0].fillna(0).replace('','0').astype(int)

In [ ]:
#年月

built_year = years.str.replace('\n      ','')
built_month = built_year.str.split('年', expand=True)[1]

built_year = built_year.str.split('年', expand=True)[0]
built_year = built_year.fillna(0).replace('NA',0).astype(int)

built_month = built_month.str.split('月', expand=True)[0]
built_month = built_month.fillna(0).astype(int)

In [ ]:
#戸数
room_total = totals.str.split('戸', expand=True)[0]
room_total = room_total.replace('NA',0).replace('',0)

In [ ]:
levels = floors.str.replace('\n      ','')
floor_levels = levels[levels.str.contains('階部').fillna(False)].str.split('階', expand=True)[0].str.replace('地下','-').astype(int)
test = levels[levels.str.contains('階建').fillna(False)].str.split('/', expand=True)
test2 = test[0].where(test[0].str.contains('階建'),test[0])
floor1 = test2[test2.str.contains('階建')].str.split('階', expand=True)[0].astype(int)
floor2 = test[1][test[1].str.contains('階建').fillna(False)].str.split('階', expand=True)[0].astype(int)
floors = pd.concat([floor1,floor2]).sort_index()


In [ ]:
df = pd.concat([names, room_types, prices_int, grosses_float, places_rev, route, station_rev,walk_mins,bus, built_year, built_month, room_total, stractures,area_float,floor_levels, floors],axis=1)

In [ ]:
df.columns = ['物件名','間取り','価格（千円）','表面利回り','住所','路線','最寄り駅','徒歩（分）','バス（分）','築年','築月','建物総戸数','造り','専有面積（平米）','階数','総階数']

In [ ]:
df.to_csv("20180317rakumachi.csv", encoding='utf-8')

In [ ]:
df

In [23]:
df

,物件名,間取り,価格（千円）,表面利回り,住所,路線,最寄り駅,徒歩（分）,バス（分）,築年,築月,建物総戸数,造り,専有面積（平米）,階数,総階数
0,■都営浅草線「浅草」駅 徒歩4分■,1R,14000,0.0601,東京都台東区浅草１丁目,都営浅草線,浅草駅,4,0,1983,1,47,SRC造,16.99,3.0,9.0
1,■都営浅草線「浅草」駅 徒歩4分■,1R,14000,0.0601,東京都台東区浅草１丁目,都営浅草線,浅草駅,4,0,1983,1,47,SRC造,16.99,3.0,9.0
2,■JR総武・中央緩行線「中野」駅 徒歩8分■,1R,19900,0.0512,東京都中野区中野５丁目,JR総武・中央緩行線,中野駅,8,0,1993,3,6,RC造,29.95,2.0,3.0
3,■都営大江戸線「東新宿」駅 徒歩7分■,1R,13800,0.0575,東京都新宿区新宿７丁目,都営大江戸線,東新宿駅,7,0,1991,9,19,RC造,21.61,5.0,7.0
4,都営大江戸線「若松河田」駅 徒歩8分■,1R,13800,0.0652,東京都新宿区新宿７丁目,都営大江戸線,若松河田駅,8,0,1994,9,14,RC造,19.02,3.0,4.0
5,ワコーレ亀有Ⅱ,1R,9300,0.0761,東京都足立区東和2-12-17,JR千代田・常磐緩行線,亀有駅,10,0,1989,5,29,RC造,16.80,7.0,7.0
6,■JR山手線「渋谷」駅 徒歩5分■,1R,38500,0.0000,東京都渋谷区円山町,JR山手線,渋谷駅,5,0,2004,2,218,SRC造,30.97,13.0,14.0
7,■JR山手線「新宿」駅 徒歩10分■,1R,15330,0.0618,東京都新宿区西新宿３丁目５−１２,JR山手線,新宿駅,10,0,1980,1,158,SRC造,22.80,11.0,11.0
8,■東急目黒線「武蔵小山」駅 徒歩8分■,1R,20700,0.0473,東京都品川区荏原４丁目,東急目黒線,西小山駅,8,0,2001,5,28,RC造,23.05,2.0,6.0
9,■JR山手線「高田馬場」駅 徒歩8分■,1R,15800,0.0000,東京都新宿区高田馬場３丁目,JR山手線,高田馬場駅,8,0,1970,7,73,SRC造,26.42,5.0,11.0
